<a href="https://colab.research.google.com/github/GergelyTuri/sleep/blob/master/tsclustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First Part (Python):

In [28]:
import pandas as pd
import numpy as np
from os.path import join
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## Data
dfof: d_awake, d_sleep \\
cosine similarity: ts_dist_awake, ts_dist_sleep



In [29]:
summary_new = pd.read_csv("/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/Analysis/summary_sleep.csv")
data_path = '/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/Analysis/data/5HT2afl05b_1/7_12_SD2'
file_name = 'dfof_mc.csv'
dfof_data = pd.read_csv(join(data_path, file_name))
dfof_data.set_index('roi_label', drop=True, inplace=True)

In [38]:
def process_dfof_mc(dfof, summary_new):
    n_roi = dfof.shape[0]
    n_time = dfof.shape[1]
    awake = []
    sleep = []
    for j in range(1, 21, 2):  # Adjusted range for Python's 0-indexing
        awake.extend(range(summary_new['start'][j-1], summary_new['end'][j-1]+1))
        sleep.extend(range(summary_new['start'][j], summary_new['end'][j]+1))

    d_awake = dfof.iloc[:, [x-1 for x in awake]]  # Adjust index for Python
    d_sleep = dfof.iloc[:, [x-1 for x in sleep]]
    return {'d_awake': d_awake, 'd_sleep': d_sleep}

In [42]:
result = process_dfof_mc(dfof_data, summary_new)
d_awake = result['d_awake']
d_sleep = result['d_sleep']

In [46]:
from scipy.spatial.distance import cosine
# ts_dist_awake
X = np.array(d_awake)
n = X.shape[0]
dist = np.empty((n, n))
dist.fill(np.nan)

for i in range(n - 1):
    for j in range(i + 1, n):
        dist[i, j] = 1 - cosine(X[i, :], X[j, :])

# Fill the lower triangular part of the matrix to make it symmetric, since cosine distance is symmetric
for i in range(n):
    for j in range(i + 1, n):
        dist[j, i] = dist[i, j]

min_val = np.nanmin(dist)
max_val = np.nanmax(dist)
dist_awake = (dist - min_val) / (max_val - min_val)

# Convert the numpy array to a pandas DataFrame for saving it as CSV
ts_dist_awake = pd.DataFrame(dist_awake)
ts_dist_awake.to_csv('/gdrive/Shareddrives/Turi_lab/Data/Sleep/ts_dist_awake.csv', index=False)

# ts_dist_sleep
X = np.array(d_sleep)
n = X.shape[0]
dist = np.empty((n, n))
dist.fill(np.nan)

for i in range(n - 1):
    for j in range(i + 1, n):
        dist[i, j] = 1 - cosine(X[i, :], X[j, :])

# Fill the lower triangular part of the matrix to make it symmetric, since cosine distance is symmetric
for i in range(n):
    for j in range(i + 1, n):
        dist[j, i] = dist[i, j]

min_val = np.nanmin(dist)
max_val = np.nanmax(dist)
dist_sleep = (dist - min_val) / (max_val - min_val)

# Convert the numpy array to a pandas DataFrame for saving it as CSV
ts_dist_sleep = pd.DataFrame(dist_sleep)
ts_dist_sleep.to_csv('/gdrive/Shareddrives/Turi_lab/Data/Sleep/ts_dist_sleep.csv', index=False)

Get significant cell:

In [49]:
pip install neuropil_analysis

ERROR: Could not find a version that satisfies the requirement neuropil_analysis (from versions: none)
ERROR: No matching distribution found for neuropil_analysis


In [47]:
import sys
from os.path import join
import matplotlib.pyplot as plt
sys.path.append('/content/sleep/notebooks/neuropil_analysis/')
import neuropil as npil
sys.path.append("/content/sleep/src/")
from eeg_class import eegData
from behavior_class import behaviorData

#Loading sleep and other behavior data (velocity)
sleep_data = pd.read_csv(join(data_path, 'velo_eeg.csv'))
#Determining awake mobile and imobile periods
sleep_data['mobile_immobile'] = behaviorData.define_immobility(
        sleep_data["filtered velo"])
#Creating brain state filters for filtering the datasets
brain_states =  eegData.brain_state_filter(
        velo_eeg_df=sleep_data,
        states=["awake_mobile", "awake_immobile", "NREM", "REM", "other"],
    )
nrem_mask = brain_states['NREM']
awake_mask = brain_states['awake_immobile']
nrem_intervals = dfof_data.loc[:, nrem_mask.values]
awake_intervals = dfof_data.loc[:, awake_mask.values]
#Calculating significant cells
significant_cells_nrem_vs_awake = npil.statistics.significance_calc(nrem_intervals.to_numpy(),
                                                      awake_intervals.to_numpy())
significant_cells_awake_vs_nrem = npil.statistics.significance_calc(awake_intervals.to_numpy(),
                                                      nrem_intervals.to_numpy())
sig_cell_mask = np.zeros(dfof_data.shape[0], dtype=bool)
sig_cell_mask[significant_cells_nrem_vs_awake] = True
significant_cell_data = dfof_data[sig_cell_mask]
significant_cell_data

ModuleNotFoundError: No module named 'neuropil'

Second Part (R):

In [ ]:
ts_dist_awake = read.csv("/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/ts_dist_awake.csv")
ts_dist_sleep = read.csv('/gdrive/Shareddrives/Turi_lab/Data/Sleep/ts_dist_sleep.csv')

net.epsilon.create <- function(dist, epsilon) {
    n = matrix(0, ncol(dist), nrow(dist))
    n[dist < epsilon] = 1;
    return(graph.adjacency(n, mode="undirected", diag=F));
}

In [ ]:
eps_seq = seq(0.01,0.99,0.01)
ddd = function(ts_dist){
modu = c()
n_hubs = c()
n_edges = c()
for (i in 1:length(eps_seq)){
net1 = net.epsilon.create(dist = ts_dist, epsilon = eps_seq[i])
c1 = cluster_fast_greedy(net1)
hub_score1 = hub.score(net1)
n_hubs[i] = length(which(hub_score1[["vector"]]>0.9))
modu[i] = modularity(c1)
n_edges[i] = gsize(net1)
}
return(list(n_hubs,n_edges,modu))
}

tt = tibble(
  eps = rep(eps_seq,2),
  state = c(rep("aw",length(eps_seq)),rep("sl",length(eps_seq))),
  n_hub = c(ddd(ts_dist_awake)[[1]],ddd(ts_dist_sleep)[[1]]),
  n_edge = c(ddd(ts_dist_awake)[[2]],ddd(ts_dist_sleep)[[2]]),
  modu = c(ddd(ts_dist_awake)[[3]],ddd(ts_dist_sleep)[[3]])
)

tt %>%
  ggplot(aes(x=eps,y=modu,colour=state))+
  geom_point(size=0.8)+
  geom_line()

tt %>%
  ggplot(aes(x=eps,y=n_hub,colour=state))+
  geom_point(size=0.8)+
  geom_line()

tt %>%
  ggplot(aes(x=eps,y=n_edge,colour=state))+
  geom_point(size=0.8)+
  geom_line()

tt %>%
  ggplot(aes(x=n_edge,y=n_hub,colour=state))+
  geom_point(size=0.8)+
  geom_line()

## significant sleep Gergo
tibble(
  n = 1:length(rownames(roi_cluster)),
  roi = rownames(roi_cluster)) %>%
  filter(roi %in% sig_cell) %>% view


In [ ]:
ts_dist = ts_dist_sleep
eps = 0.2
net = net.epsilon.create(dist = ts_dist, epsilon = eps)
c = cluster_fast_greedy(net)
membership(c)
plot(net, vertex.color=membership(c), layout=layout_with_fr(net))
#hub score
hub_score <- hub.score(net)
which(hub_score[["vector"]]>=0.9)